In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 5GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import pandas_profiling
import warnings
import collections
warnings.filterwarnings("ignore")
import plotly
import plotly.express as px
from plotly.subplots import make_subplots


In [ ]:
pip install ggplot

In [ ]:
import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

In [ ]:
tv_shows = pd.read_csv('/kaggle/input/tv-shows-on-netflix-prime-video-hulu-and-disney/tv_shows.csv')
tv_shows.head()

In [ ]:
tv_shows.info()

In [ ]:
tv_shows.shape

In [ ]:
plt.figure(figsize=(12, 5))
plt.hist(tv_shows['IMDb'].values, bins=250)
plt.title('IMDb rating')
plt.xlabel('Value')
plt.ylabel('Count')
plt.show()

First we remove the unwanted columns in the process of data cleaning.

we will drop duplicates based on title

In [ ]:
tv_shows.drop_duplicates(subset='Title',keep='first',inplace=True)

# **Feature Scaling**
First fill the novalues/NaN with zeros and convert both of them into integers.
Convert both ratings IMDB and Rotten Tomatoes on common scale ie. on 100.

In [ ]:
tv_shows['Rotten Tomatoes'] = tv_shows['Rotten Tomatoes'].fillna('0%')
tv_shows['Rotten Tomatoes'] = tv_shows['Rotten Tomatoes'].apply(lambda x : x.rstrip('%'))
tv_shows['Rotten Tomatoes'] = pd.to_numeric(tv_shows['Rotten Tomatoes'])

tv_shows['IMDb'] = tv_shows['IMDb'].fillna(0)
tv_shows['IMDb'] = tv_shows['IMDb']*10
tv_shows['IMDb'] = tv_shows['IMDb'].astype('int')

In [ ]:
tv_shows['IMDb'].head()

In [ ]:
tv_shows['Rotten Tomatoes'].head()

here we converted both the ratings into same scale on 100.

# **Wide to Long Format Conversion**
Plotting of the charts would be easier if we get the 1s and 0s in the columns Netflix,Hulu,Disney+ & Prime Video under a categorical section videos, also there might be cases wherin the same show is present in more than one service.

In [ ]:
tv_shows_long=pd.melt(tv_shows[['Title','Netflix','Hulu','Disney+','Prime Video']],id_vars=['Title'],var_name='StreamingOn', value_name='Present')
tv_shows_long = tv_shows_long[tv_shows_long['Present'] == 1]
tv_shows_long.drop(columns=['Present'],inplace=True)

In [ ]:
tv_shows.head()

now merge the netflix,prime,hulu and desney into one column, which has categorical data. 

In [ ]:
tv_shows_merged = tv_shows_long.merge(tv_shows, on='Title', how='inner')

In [ ]:
tv_shows_merged.drop(columns = ['Unnamed: 0','Netflix','Hulu', 'Prime Video', 'Disney+', 'type'], inplace=True,axis = 1)


In [ ]:
tv_shows_merged.head()

Firstly to get the Service with the most content

In [ ]:
tv_shows_merged.groupby('StreamingOn').Title.count().plot(kind='bar')\

Scatter plot between IMDB and Rotten tomatoes ratings to get the streaming service that has best of both worlds.

In [ ]:
tv_shows_ratings = tv_shows_merged[(tv_shows_merged.IMDb > 0) & tv_shows_merged['Rotten Tomatoes'] > 0]
px.scatter(tv_shows_ratings, x='IMDb',y='Rotten Tomatoes',color='StreamingOn')

In [ ]:
corr = tv_shows_ratings.corr(method='kendall')
plt.figure(figsize=(14,7))
sns.heatmap(corr, annot=True)